# Idea
Create Dataset from ChatGPT

In [1]:
import os
from pathlib import Path
import glob
import pandas as pd

import numpy as np

In [2]:
os.chdir("..")
path = Path("data/GPTCloneBench/standalone/true_semantic_clones/py")
assert path.exists(), "Path does not exist"

#### Erronous:
data/GPTCloneBench/standalone/true_semantic_clones/py/prompt_2/T4/Clone_4245.py (more than 2 functions)
data/GPTCloneBench/standalone/true_semantic_clones/py/prompt_2/T4/Clone_1287.py (less than 3 newlines in between)
data/GPTCloneBench/standalone/true_semantic_clones/py/prompt_2/T4/Clone_2448.py (no new lines inbetween)
data/GPTCloneBench/standalone/true_semantic_clones/py/prompt_2/T4/Clone_1414.py is syntactically wrong and also clone 3 => deleted
 data/GPTCloneBench/standalone/true_semantic_clones/py/prompt_2/T4/Clone_3439.py (less than 3 newlines in between)
 data/GPTCloneBench/standalone/true_semantic_clones/py/prompt_2/T4/Clone_1415.py (less than 3 newlines in between)
 data/GPTCloneBench/standalone/true_semantic_clones/py/prompt_2/T4/Clone_2446.py (no new lines inbetween)
 data/GPTCloneBench/standalone/true_semantic_clones/py/prompt_2/T4/Clone_3441.py (less than 3 newlines in between)
 data/GPTCloneBench/standalone/true_semantic_clones/py/prompt_2/T4/Clone_1413.py (less than 3 newlines in between)
 data/GPTCloneBench/standalone/true_semantic_clones/py/prompt_2/T4/Clone_1288.py (less than 3 newlines in between)

In [3]:
import re

python_files = path.glob("**/*.py")
pattern = re.compile(r"data/GPTCloneBench/standalone/true_semantic_clones/py/(?P<prompt>.*?)/(?P<clonetype>.*?)/Clone_(?P<id>\d+)\.py")
split_pattern = re.compile(r"\n\s{0,4}\n\s?\n([#\w=\s:.\-,]*\n)?\n?\s{0,6}(def|from|import|mutex|depth|@patch|BUCKET|k)")

clones = []
for file in python_files:
    match = pattern.match(str(file))
    if match:
        groups_dict = dict(match.groupdict())
        
        if groups_dict["clonetype"] == "MT3":
            continue
        
        with open(file) as f:
            content = f.read()
        splits = split_pattern.split(content, maxsplit=1)
        assert len(splits) == 4, print(f"no match for {file}: {len(splits)}: {splits}")
            
        clone1 = splits[0]
        clone2 = splits[2] +" "+ splits[3]
        
        groups_dict["clone1"] = clone1.strip().strip("\n")
        groups_dict["clone2"] = clone2.strip().strip("\n")
        groups_dict["file_path"] = file

        clones.append(groups_dict)
    else:
        print(f"no match for {file}")

In [4]:
df = pd.DataFrame(clones)
# df.set_index("id", inplace=True)
df.sort_values(by="id",inplace=True)

#### Checking Data

In [5]:
df["clonetype"].value_counts()

clonetype
T4    6115
Name: count, dtype: int64

In [6]:
method_signature_pattern_at_start = re.compile(r'^def\s{1,2}\w+\s*\([\w,=\s*.\'\"-{}()\[\]]*\)\s*:\s*') 

def starts_with_method_signature(s):
    if method_signature_pattern_at_start.match(s) or s.startswith("from") or s.startswith("import") or s.startswith("mutex") or s.startswith("depth") or s.startswith("@patch") or s.startswith("BUCKET") or s.startswith("k"):
        return True
    return False
df[~df["clone1"].map(lambda x: starts_with_method_signature(x))]

,prompt,clonetype,id,clone1,clone2,file_path


In [7]:
df[~df["clone2"].map(lambda x: starts_with_method_signature(x))]

,prompt,clonetype,id,clone1,clone2,file_path
103,prompt_2,T4,3608,"def __init__(self, maxlen, items = None) :\n\t...","def __init__(self, maxlen, items=None) -> Non...",data/GPTCloneBench/standalone/true_semantic_cl...
1223,prompt_2,T4,4392,"def merge(left_side, right_side) :\n\tresult =...","def merge(left: List[int], right: List[int]) ...",data/GPTCloneBench/standalone/true_semantic_cl...
5988,prompt_1,T4,463,"def pairsum_n(list1, value) :\n\tset1 = set(li...","def pairsum_n(list1: List[int], value: int) -...",data/GPTCloneBench/standalone/true_semantic_cl...
5862,prompt_1,T4,464,"def pairsum_n(list1, value) :\n\tset1 = set(li...","def pairsum_n(list1: List[int], value: int) -...",data/GPTCloneBench/standalone/true_semantic_cl...
675,prompt_2,T4,4897,"def recursiveHalfString(s, offset = 0) :\n\tha...","def recursiveHalfString(s: str, offset: int =...",data/GPTCloneBench/standalone/true_semantic_cl...
3159,prompt_2,T4,4898,"def recursiveHalfString(s, offset = 0) :\n\tha...","def recursiveHalfString(s: str, offset: Optio...",data/GPTCloneBench/standalone/true_semantic_cl...
4016,prompt_2,T4,4902,"def recursiveHalfString(s, offset = 0) :\n\tha...","def recursiveHalfString(s: str, offset = 0, h...",data/GPTCloneBench/standalone/true_semantic_cl...
4760,prompt_2,T4,5362,"def __init__(self, widget) :\n\tlogging.Handle...","def ault_fg = ""red""\n for tag in [""INFO"", ""...",data/GPTCloneBench/standalone/true_semantic_cl...


#### Pairing

In [8]:
valcnt = pd.DataFrame(df["clone1"].value_counts(), index=None)
valcnt.reset_index(drop=False, inplace=True)
valcnt.head()

,clone1,count
0,def median(data) :\n\tnew_list = sorted(data)\...,73
1,"def merge(left_side, right_side) :\n\tresult =...",55
2,"def sublist(a, b) :\n\tlast = 0\n\tfor el_a in...",48
3,def unique(list) :\n\ts = {}\n\toutput = []\n\...,48
4,def median(x) :\n\tif len(x) % 2 == 0 :\n\t\tx...,42


In [9]:
import pandas as pd

def split_dataframe(df):
    total_sum = df['count'].sum()
    target_sum = total_sum / 2
    
    part1 = []
    part2 = []
    
    current_sum = 0
    add_to_part1 = True
    
    for index, row in df.iterrows():
        if add_to_part1:
            part1.append(row)
            current_sum += row['count']
        else:
            part2.append(row)
        
        add_to_part1 = not add_to_part1
    
    return pd.DataFrame(part1, columns=df.columns), pd.DataFrame(part2, columns=df.columns)

part1, part2 = split_dataframe(valcnt)

part1["count"].sum(), part2["count"].sum()

(3075, 3040)

In [10]:
df1 = df[df["clone1"].isin(part1["clone1"])]
df2 = df[df["clone1"].isin(part2["clone1"])]
df1.shape, df2.shape

((3075, 6), (3040, 6))

In [18]:
df1[df1["clone1"].isin(df2["clone1"])]
df2[df2["clone1"].isin(df1["clone1"])]

,prompt,clonetype,id,clone1,clone2,file_path


In [20]:
pairs = list(zip(df1["clone1"], df2["clone2"]))
print("All pairs",len(pairs))
# pairs[:2]
print("Dissimilar pairs", len(set(pairs)))

All pairs 3040
Dissimilar pairs 3021


In [22]:
pairs2 = list(zip(df1["clone2"], df2["clone1"]))
print("All pairs",len(pairs2))
# pairs[:2]
print("Dissimilar pairs", len(set(pairs2)))

All pairs 3040
Dissimilar pairs 3027


In [26]:
# TODO: CHECK WHY DUBLICATES !!! (maybe check old dataframe)
disimmilar_pairs_df = pd.DataFrame(pairs+pairs2, columns=['clone1', 'clone2'])

In [27]:
disimmilar_pairs_df[disimmilar_pairs_df.duplicated()]

,clone1,clone2
55,"def split_at_first_false(pred, seq) :\n\tindex...",def almostIncreasingSequence(sequence): \n\tf...
239,def cumulative_sum(lst):\n new_lst = []\n ...,def curry(func):\n def call_with(*args):\n ...
242,"def convert(items, ID) :\n\tfor key, value in ...",def curry(func):\n def call_with(*args):\n ...
244,"def convert(items, ID) :\n\tfor key, value in ...",def curry(func):\n def call_with(*args):\n ...
474,"def char_first_index(s, c) :\n\tif len_rec(s) ...","def __call__(self, new) :\n\t if new: \n\t\..."
1366,"def traceit(frame, event, arg) :\n\tif event =...","def find_nth(string, substring, n):\n coun..."
1379,def checkio(data) :\n\tfor index in range(len(...,"def touch_value(self, stack, frame) :\n\tasse..."
1443,"def recurse(left, right, threshold, features, ...",def get_cost(x):\n duration = x['t1']\n ...
1516,def all_pairs(lst) :\n\tif not lst :\n\t\tyiel...,"def binary_insert(root, node):\n if not ro..."
2041,"def merge(left, right) :\n\tresult = []\n\ti =...","def __setattr__(self, key, value):\n if ke..."


Merging

In [30]:
disimmilar_pairs_df.shape

(6080, 2)

In [29]:
df.shape

(6115, 6)

In [37]:
df["semantic_clone"]=1
disimmilar_pairs_df["semantic_clone"]=0

df_merge = pd.concat([df, disimmilar_pairs_df], ignore_index=True)

In [39]:
df_merge = df_merge[["clone1", "clone2", "semantic_clone"]]

In [40]:
df_merge

# df_merge.to_csv("data/gpt_benchmark_dataset.csv", index=False)

,clone1,clone2,semantic_clone
0,def second_largest(numbers) :\n\tcount = 0\n\t...,def second_largest(numbers) :\n\tsecond_large...,1
1,def second_largest(numbers) :\n\tcount = 0\n\t...,def second_largest(numbers):\n\n if len(numb...,1
2,"def num_input(prompt, error) :\n\twhile True :...","def num_input(prompt, error) :\n while...",1
3,"def recurse(node, depth) :\n\tindent = "" "" * ...","def recurse(node, depth) :\n\tindent = "" "" *...",1
4,"def getName(self, name) :\n\tlisty = []\n\tspl...","def getName2(self, fullname) :\n\tnames = []\...",1
...,...,...,...
12190,"def ignore_until(the_list, match):\n sub_l...",def run(self) :\n\tt = datetime(* datetime.now...,0
12191,"def ignore_until(the_list, match):\n sub_l...","def __init__(self, year = None, month = None,d...",0
12192,"def ignore_until(the_list, match):\n end_i...","def __init__(self, year = None, month = None,d...",0
12193,"def ignore_until(the_list, match): \n idx ...","def __init__(self, year = None, month = None,d...",0


#### Removing Method Names

In [46]:
method_name_pattern = re.compile(r"def\s+(.+?)\(")
df_rename = df_merge.copy()

df_rename["clone1"] = df_rename["clone1"].map(lambda x: method_name_pattern.sub("def method_name(", x.strip().strip("\n"), count=1))
df_rename["clone2"] = df_rename["clone2"].map(lambda x: method_name_pattern.sub("def method_name(", x.strip().strip("\n"), count=1))
df_rename.shape

(12195, 3)

In [49]:
df_rename["clone1"].str.count("method_name").sum(), df_rename["clone2"].str.count("method_name").sum()

(12195, 12195)

In [50]:
df_rename.to_csv("data/gpt_benchmark_dataset.csv", index=False)